In [47]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. Dane przykładowe
# Załóżmy, że mamy 3 klasy i batch size = 4
logits = torch.tensor([[2.0, 0.5, -1.0],
                       [0.1, 0.2, 0.7],
                       [1.2, -0.3, 0.1],
                       [0.0, 1.0, 0.5]], requires_grad=True)
# Etykiety (ground-truth)
targets = torch.tensor([0, 2, 1, 1], dtype=torch.long)

# 2. Definiujemy warstwę LogSoftmax + NLLLoss
log_softmax = nn.LogSoftmax(dim=1)   # zamiana logits → log-probs
criterion = nn.NLLLoss(reduction='mean')

# 3. Forward pass
log_probs = log_softmax(logits)      # kształt: (4, 3)
log_probs

tensor([[-0.2413, -1.7413, -3.2413],
        [-1.3679, -1.2679, -0.7679],
        [-0.4421, -1.9421, -1.5421],
        [-1.6803, -0.6803, -1.1803]], grad_fn=<LogSoftmaxBackward0>)

In [48]:
preds = torch.argmax(log_probs, dim=1)
preds

tensor([0, 2, 0, 1])

In [49]:
N = log_probs.size(0)
N

4

In [50]:
torch.arange(N)

tensor([0, 1, 2, 3])

In [51]:
log_probs[torch.arange(N), targets]

tensor([-0.2413, -0.7679, -1.9421, -0.6803], grad_fn=<IndexBackward0>)

In [52]:
selected_log_probs = log_probs[torch.arange(N), targets]
selected_log_probs

tensor([-0.2413, -0.7679, -1.9421, -0.6803], grad_fn=<IndexBackward0>)

In [53]:
neg_log_likelihood = -selected_log_probs
neg_log_likelihood

tensor([0.2413, 0.7679, 1.9421, 0.6803], grad_fn=<NegBackward0>)

In [54]:
neg_log_likelihood.mean()

tensor(0.9079, grad_fn=<MeanBackward0>)

In [55]:

loss = criterion(log_probs, targets)

print(f"Log-prawdopodobieństwa:\n{log_probs.detach()}\n")
print(f"Wartość straty: {loss.item():.4f}")

# 4. Backward i update (przykład uproszczony)
loss.backward()
with torch.no_grad():
    # przykładowa prosta aktualizacja gradientowa
    for param in [logits]:
        param -= 0.1 * param.grad

Log-prawdopodobieństwa:
tensor([[-0.2413, -1.7413, -3.2413],
        [-1.3679, -1.2679, -0.7679],
        [-0.4421, -1.9421, -1.5421],
        [-1.6803, -0.6803, -1.1803]])

Wartość straty: 0.9079
